In [ ]:
import os
import re
import sys
import pickle
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

if '..' not in sys.path:
    sys.path.append('..')
from pfcommon import build_network_graph, make_full_object_name, get_objects, Node, Edge

In [ ]:
data_file = 'nine-bus_graph'
data_file = 'sardinia_graph'
force = True

if force or not os.path.isfile(data_file + '.pkl'):
    powerfactory_path = r'C:\Program Files\DIgSILENT\PowerFactory 2024 SP1\Python\3.10'
    if powerfactory_path not in sys.path:
        sys.path.append(powerfactory_path)
    import powerfactory as pf

    if data_file == 'nine-bus_graph':
        project_name = '\\Terna_Inerzia\\Nine-bus System'
    elif data_file == 'sardinia_graph':
        project_name = '\\Terna_Inerzia\\V2020_Rete_Sardegna_2021_06_03cr'
    else:
        raise Exception('Unknown data file name')

    app = pf.GetApplication()
    if app is None:
        raise Exception('Cannot get PowerFactory application')
    print('Got PowerFactory application.')
    
    err = app.ActivateProject(project_name)
    if err:
        raise Exception(f'Cannot activate project {project_name}')
    print(f'Activated project "{project_name}".')
    
    graph,edges,nodes = build_network_graph(app, verbose=True)

    data = {
        'graph': graph,
        'edges': edges,
        'nodes': nodes
    }
    pickle.dump(data, open(data_file + '.pkl', 'wb'))
    
else:
    data = pickle.load(open(data_file + '.pkl', 'rb'))
    graph = data['graph']
    edges = data['edges']
    nodes = data['nodes']

In [ ]:
G = nx.MultiGraph()
for e in edges:
    G.add_edge(e.node1.name, e.node2.name, weight=e.length, label=e.name, voltage=e.voltage)
voltages = np.unique([e.voltage for e in edges])[::-1]
lengths = np.unique([e.length for e in edges])[::-1]

bare_G = nx.MultiGraph()
bare_G.add_edges_from([(e.node1.name,e.node2.name) for e in edges])

n_nodes = len(G.nodes)
n_edges = len(G.edges)
n_cc = nx.number_connected_components(G)
print('The graph is{} connected.'.format('' if nx.is_connected(G) else ' not'))
print('No. of nodes: {}'.format(n_nodes))
print('No. of edges: {}'.format(n_edges))
print('No. of connected components: {}'.format(n_cc))
print('Voltage levels: [{}] kV'.format(', '.join(map(lambda w: f'{w:g}', voltages))))

In [ ]:
layout_algo = 'kamada_kawai'
weighted_pos = False
pos_file = data_file + '_' + layout_algo + '_layout'
pos_file += '_weighted.pkl' if weighted_pos else '.pkl'
force = False
if not os.path.isfile(pos_file) or force:
    print('Output file: {}'.format(pos_file))
    sys.stdout.write('Computing graph layout... ')
    sys.stdout.flush()
    if layout_algo == 'kamada_kawai':
        # takes forever using the weighted graph
        pos = nx.kamada_kawai_layout(G if weighted_pos else bare_G)
    elif layout_algo == 'spring':
        pos = nx.spring_layout(G if weighted_pos else bare_G, seed=1983)
    else:
        raise Exception('Unknown layout algorithm: `{}`'.format(layout_algo))
    sys.stdout.write('done.\n')
    pickle.dump(pos, open(pos_file, 'wb'))
else:
    print('Input file: {}'.format(pos_file))
    pos = pickle.load(open(pos_file, 'rb'))

In [ ]:
Vmin = 12
cmap = plt.get_cmap('rainbow_r', (voltages>=Vmin).sum())
fig,ax = plt.subplots(1, 1, figsize=(12,12))
# labels = {n: n.split('|')[-1].split('__')[0] for n in graph.nodes if 'FSACT' in n}
# nx.draw_networkx_labels(graph, pos, labels=labels, font_size=6, ax=ax)
for i,V in enumerate(voltages):
    if V >= Vmin:
        idx = [(a,b) for a,b,d in G.edges(data=True) if d['voltage'] == V]
        lbl = '{:g} kV'.format(V)
    else:
        idx = [(a,b) for a,b,d in G.edges(data=True) if d['voltage'] <= V]
        lbl = r'$\leq{:g}$ kV'.format(V)
#     wdt = np.log(V) / np.log(7)
    wdt = V / 70
    nx.draw_networkx_nodes(G.subgraph([jdx[0] for jdx in idx]), pos,
                           node_color='k', node_size=wdt*2, ax=ax)
    nx.draw_networkx_nodes(G.subgraph([jdx[1] for jdx in idx]), pos,
                           node_color='k', node_size=wdt*2, ax=ax)
    nx.draw_networkx_edges(G, pos, edgelist=idx, width=wdt, edge_color=cmap(i), label=lbl, ax=ax)
    if V < Vmin:
        break
ax.axis('off')
ax.axis('equal')
ax.legend(loc='upper left', frameon=False, fontsize=8)
fig.tight_layout()
plt.savefig(data_file + '.pdf')